In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.types import *
from pyspark.sql.functions import *

In [2]:
spark = SparkSession.builder \
.master("local") \
.appName("join_on_multiple_columns_df") \
.getOrCreate()

In [3]:
emp_data = [(1,"Smith",-1,"2018",10,"M",3000),
    (2,"Rose",1,"2010",20,"M",4000),
    (3,"Williams",1,"2010",10,"M",1000),
    (4,"Jones",2,"2005",10,"F",2000),
    (5,"Brown",2,"2010",30,"",-1),
    (6,"Brown",2,"2010",50,"",-1)
      ]
empColumns = ("emp_id","name","superior_emp_id","branch_id","dept_id","gender","salary")
emp = spark.createDataFrame(data=emp_data, schema=empColumns)

In [4]:
dept_data = [("Finance",10,"2018"),
    ("Marketing",20,"2010"),
    ("Marketing",20,"2018"),
    ("Sales",30,"2005"),
    ("Sales",30,"2010"),
    ("IT",50,"2010")
            ]

deptColumns = ("dept_name","dept_id","branch_id")
dept = spark.createDataFrame(data=dept_data, schema=deptColumns)

In [13]:
# Using multiple columns on join expression
emp.join(dept,(emp.dept_id == dept.dept_id) & (emp.branch_id == dept.branch_id),"inner").show()

+------+-----+---------------+---------+-------+------+------+---------+-------+---------+
|emp_id| name|superior_emp_id|branch_id|dept_id|gender|salary|dept_name|dept_id|branch_id|
+------+-----+---------------+---------+-------+------+------+---------+-------+---------+
|     5|Brown|              2|     2010|     30|      |    -1|    Sales|     30|     2010|
|     2| Rose|              1|     2010|     20|     M|  4000|Marketing|     20|     2010|
|     6|Brown|              2|     2010|     50|      |    -1|       IT|     50|     2010|
|     1|Smith|             -1|     2018|     10|     M|  3000|  Finance|     10|     2018|
+------+-----+---------------+---------+-------+------+------+---------+-------+---------+



In [14]:
emp.createOrReplaceTempView("employee")
dept.createOrReplaceTempView("department")
spark.sql("""select e.*, d.* from employee e inner join department d on e.dept_id=d.dept_id
and e.branch_id=d.branch_id""").show()

+------+-----+---------------+---------+-------+------+------+---------+-------+---------+
|emp_id| name|superior_emp_id|branch_id|dept_id|gender|salary|dept_name|dept_id|branch_id|
+------+-----+---------------+---------+-------+------+------+---------+-------+---------+
|     5|Brown|              2|     2010|     30|      |    -1|    Sales|     30|     2010|
|     2| Rose|              1|     2010|     20|     M|  4000|Marketing|     20|     2010|
|     6|Brown|              2|     2010|     50|      |    -1|       IT|     50|     2010|
|     1|Smith|             -1|     2018|     10|     M|  3000|  Finance|     10|     2018|
+------+-----+---------------+---------+-------+------+------+---------+-------+---------+

